In [ ]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import random
import numpy

import matplotlib.pyplot as plt
import seaborn as sns

import statistics

In [ ]:
# problem constants
ONE_MAX_LENGTH = 100 # length of bit string to be optimized

# Genetic Algorithm constants:
POPULATION_SIZE = 200 # number of individuals in population
P_CROSSOVER = 0.9 # probability for crossover
P_MUTATION = 0.1 # probability for mutating and individual
MAX_GENERATIONS = 50 # stopping condition
HALL_OF_FAME_SIZE = 10 # fixed best individuals

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
# configure statistics
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", numpy.max)
stats.register("avg", numpy.mean)

In [ ]:
# configure elitism
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

In [ ]:
# create fitness function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


# fitness function definition
def oneMaxFitness(individual):
    return sum(individual), # return a tuple


toolbox = base.Toolbox()

# create genetic operators
toolbox.register("evaluate", oneMaxFitness)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ONE_MAX_LENGTH)

# create individual
toolbox.register("zeroOrOne", random.randint, 0, 1)
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, ONE_MAX_LENGTH)

# create population
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
population = toolbox.populationCreator(n=POPULATION_SIZE)

In [ ]:
population, logbook = algorithms.eaSimple(population, 
                                          toolbox, 
                                          cxpb=P_CROSSOVER, 
                                          mutpb=P_MUTATION, 
                                          ngen=MAX_GENERATIONS,
                                          stats=stats,
                                          halloffame=hof,
                                          verbose=True)

In [ ]:
print("Hall of Fame Individuals = ", *hof.items, sep="\n")
print("Best Ever Individual = ", hof.items[0])

In [ ]:
maxFitnessValues, meanFitnessValues = logbook.select("max", "avg")

# plot statistics:
sns.set_style("whitegrid")
plt.plot(maxFitnessValues, color='red')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generation')
plt.ylabel('Max / Average Fitness')
plt.title('Max and Average Fitness over Generations')
plt.show()